In [ ]:
from google.colab import drive
drive.mount('/content/drive')
curr_dir =  "/content/drive/MyDrive/SEGMENTATION_QAZ"
%cd "$curr_dir"

Mounted at /content/drive
/content/drive/MyDrive/SEGMENTATION_QAZ


In [ ]:
pip install xlwt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 3.9 MB/s eta 0:00:00


In [ ]:
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 5.4 MB/s eta 0:00:00


In [ ]:
import re
import os
import xlrd, xlwt
import sacrebleu
import numpy as np
import openpyxl
import time
from tqdm import tqdm

def splitting_by_words(text):
    result = re.findall(r'\w+|[^\w\s]', text)
    print('text', result)
    return result
global jurnaqs
jurnaqs = []
ending_segs = []
def sorting_endings(endings_file_name):
    endings_wb = xlrd.open_workbook(endings_file_name)
    endings_sh = endings_wb.sheet_by_index(0)
    endings = []
    ending_segs = []
    for rownum in range(endings_sh.nrows-1):
        ending = endings_sh.cell(rownum+1,0).value
        if '\ufeff' in ending:
            ending = ending.replace('\ufeff', '')
        endings.append(ending)
    #print('''ENDINGS''')
    #print(endings)
    for rownum in range(endings_sh.nrows-1):
        ending_seg1 = endings_sh.cell(rownum+1,1).value
        if '\ufeff' in ending:
            ending_seg1 = ending.replace('\ufeff', '')
        ending_segs.append(ending_seg1)

    return [endings, ending_segs]


def sorting_jurnaqs(jurnaqs_file_name):
    jurnaqs_wb = xlrd.open_workbook(jurnaqs_file_name)
    jurnaqs_sh = jurnaqs_wb.sheet_by_index(0)
    jurnaqs = []
    for rownum in range(jurnaqs_sh.nrows-1):
        jurnaq = jurnaqs_sh.cell(rownum+1,0).value
        if '\ufeff' in jurnaq:
            jurnaq = jurnaq.replace('\ufeff', '')
        jurnaqs.append(jurnaq)

    return jurnaqs

def fjurnaq_stem(word):
    rez_stem = ''
    jurnaq_stem = ''
    word_len = len(word)
    min_len_of_stem = 2
    word_len = len(word)
    if word_len > min_len_of_stem:

        n = word_len - min_len_of_stem
        i = 1  # Начинаем с 1, если индексировать с конца слова
        maxlenjurnaq = 4  # Максимальная длина суффикса (журнака)

        while i <= len(word) and i <= maxlenjurnaq:
            word_ending = word[-i:]  # Получаем окончание длиной i
            #print('jurnaq', word_ending)
            stem = word[:-i]  # Основная часть слова без окончания
            #print('stem', stem)

            for jurnaq in jurnaqs:
                if word_ending == jurnaq:
                    if stem.lower() in stems_list:
                        rez_stem = stem
                        jurnaq_stem = word_ending
                        #print('stem', rez_stem)
                        #print('jurnaq0', jurnaq_stem, stem)
                        return rez_stem, jurnaq_stem

            i += 1  # Увеличиваем i на каждой итерации
    else:
        rez_stem = word
        jurnaq_stem = ''
    #print('stem', rez_stem)
    #print('jurnaq1', jurnaq_stem, rez_stem)
    return rez_stem, jurnaq_stem


def ending_stem(word):
    word_len = len(word)
    min_len_of_word = 2
    rez_stem = ''
    rez_ending = ''
    if word_len > min_len_of_word:
        n = word_len - min_len_of_word
        if word.lower() in stems_list:
            rez_stem = word
            rez_ending = ''
            #print('stem 0', rez_stem)
            #print('ending 0', rez_ending)

            return rez_stem, rez_ending

        else:
            i = n+1
            while i > 0:
                word_ending = word[word_len - (i-1):]
                stem = word[:word_len-len(word_ending)]
                #print('stem', stem)
                #print('ending', word_ending)
                for ending in endings:
                    if word_ending == ending:
                        if stem.lower() in stems_list:
                            rez_stem = stem
                            rez_ending = word_ending
                            #print('stem 2', rez_stem)
                            #print('ending 2', rez_ending)
                            i = 0
                            break

                #print('stem 1', stem)

                rez_stem = stem
                i = i-1
    else:
        rez_stem = word
        rez_ending = ''
    return rez_stem, rez_ending


global endings_seg

def segment_word(word, endings):
    jurnaqs_seg = []
    ending_text = {}
    ending_seg_dic = {}
    endingseg = ''
    endingseg_1 = ''
    #print('word', word)
    stemm, ending = ending_stem(word)                 # вызов функции сегментирования слова на стем и окончание
    #print('stem', stemm)
    #print('ending', ending)
    stemm_len = len(stemm)

    if ending != '':
        num_ending = endings.index(ending)           # находится индекс окончание
            #print(num_ending)
        endingseg = endings_seg[num_ending]          # находится сегментированное окончание
    else:
        endingseg = ''
            #endingseg = "@@ ".join([stemm, endingseg])
            #ending_seg_dic.update({word: endingseg})
        #print('stemm1' ,stemm)                                                  # СЕГМЕНТИРОВАНИЕ СТЕМА ПО ЖУРНАКАМ
    stemm_len = len(stemm)
    min_len_of_stem = 2
    jurnaqs_seg = ''
    if stemm_len > min_len_of_stem:
        n = stemm_len - min_len_of_stem
        i = 1
        while i <= n:
            #print('stemm2', stemm)                                      # Обработка стема справа
            rez_stem, jurnaq = fjurnaq_stem(stemm)       #  вызов функции нахождения журнака справа
            #print('jurnaq_stem', jurnaq)
            if jurnaq == '':
                break
                #jurnaqs_seg.append(jurnaq_stem)

            if jurnaqs_seg != '':
                                   # Если список не пуст, соединяем последний элемент с текущим журнаком через @@
                if jurnaq != '':
                        updated_seg = f"{jurnaq}@@ {jurnaqs_seg}"
            else:
                                    # Если это первый элемент, просто добавляем текущий журнак
                updated_seg = jurnaq

                                    # Обновляем список
            jurnaqs_seg = updated_seg
                #print(jurnaqs_seg)
            stemm = rez_stem                             #  обновление стема, за счет сегментирования журнака(ов)
            i = i + len(jurnaq)
            #print('jurnaq_seg', jurnaqs_seg)


        # Формируем финальную строку: Стем + журнаки + окончания
    #print('stemm', stemm)
    #print('endingseg', endingseg)
    #print('jurnaqs_seg', jurnaqs_seg)
    if endingseg != '' and jurnaqs_seg != '':
        final_seg = f"{stemm}@@ {jurnaqs_seg}{endingseg}"
        #print('final_seg1', final_seg)
    elif endingseg != '' and jurnaqs_seg == '':
        final_seg = f"{stemm}@@ {endingseg}"
        #print('final_seg2', final_seg)
    elif endingseg == '' and jurnaqs_seg != '':
        final_seg = f"{stemm}@@ {jurnaqs_seg}"
        #print('final_seg3', final_seg)
    else:
        final_seg = f"{stemm}"
        #print('final_seg4', final_seg)


    return final_seg

def morpho_segment(text):
    tokens = re.findall(r'\w+|[^\w\s]', text)  # Слова + пунктуация
    processed_tokens = []
    #rim_cifry = ['i', 'ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix', 'x', 'xi', 'xii', 'xiii', 'xiv', 'xv', 'xvi', 'xvii', 'xviii', 'xix', 'xx', 'xxi', 'xxiv']
    for token in tokens:
        if token.lower() in stop_words:
            processed_tokens.append(token)  # Оставляем стоп-слова без изменений
        elif token.isalpha():
            #print('token', token)
            processed_tokens.append(segment_word(token, endings))  # Сегментация слов
        else:
            processed_tokens.append(token)  # Оставляем числа и знаки препинания

    return " ".join(processed_tokens)

stopwords_file_name = "stop_words.txt"
stems_file_name = "qaz_stems_unik_edit.xlsx"
jurnaqs_file_name = "qaz_jurnaks.xls"
endings_file_name = "qaz_endings_seg.xls"
endings_seg = []
endings, endings_seg = sorting_endings(endings_file_name)
jurnaqs = sorting_jurnaqs(jurnaqs_file_name)

        #------INPUT STEM FILE
stems_wb = openpyxl.load_workbook(stems_file_name)
stems_sh = stems_wb.active
stems = []
global stems_list
for row in stems_sh.rows:
    stem = row[0].value
    if '\ufeff' in stem:
        stem = stem.replace('\ufeff', '')
    stems.append(stem)
stems_list = sorted(stems, key=len, reverse=True)

         #-----INPUT STOPWORD FILE
def read_lines_with_fallback(path: str, encodings=("utf-8", "utf-8-sig", "cp1251", "cp1252", "latin-1")):
    """
    Tries multiple encodings and returns stripped non-empty lines.
    - utf-8-sig: handles BOM
    - cp1251: common for Cyrillic on Windows
    - cp1252/latin-1: last-resort fallbacks
    """
    last_err = None
    for enc in encodings:
        try:
            with open(path, "r", encoding=enc, errors="strict") as f:
                lines = f.read().splitlines()
            # Clean BOM just in case
            lines = [ln.replace("\ufeff", "").strip() for ln in lines]
            # Filter empties
            lines = [ln for ln in lines if ln]
            print(f"✅ stop_words loaded with encoding: {enc} | lines={len(lines)}")
            return lines
        except UnicodeDecodeError as e:
            last_err = e
            continue

    # If all strict attempts fail, do a safe read (won't crash, but may lose some chars)
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        lines = f.read().splitlines()
    lines = [ln.replace("\ufeff", "").strip() for ln in lines if ln.strip()]
    print("⚠️ stop_words loaded with utf-8 (errors=replace). Some characters may be replaced.")
    if last_err:
        print("Last strict decode error:", last_err)
    return lines


# -----INPUT STOPWORD FILE (FIXED)-----
stop_words = read_lines_with_fallback(stopwords_file_name)




input_path = "KAZ_TEST.txt"
output_dir = "segmented_chunks6"
chunk_size = 12


# Читаем строки
with open(input_path, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

os.makedirs(output_dir, exist_ok=True)

n_chunks = (len(lines) + chunk_size - 1) // chunk_size
print(f"📄 Всего чанков: {n_chunks}")
                  #------ END INPUT TEXT FILE AND CHANKING

for i in tqdm(range(n_chunks), desc="⏳ Сегментирую"):
    # Путь для сохранённого чанка
    output_path = os.path.join(output_dir, f"chunk_{i:03d}.txt")

    # Проверяем: если файл уже существует, пропускаем
    if os.path.exists(output_path):
        print(f"⚡ Чанк {i} уже существует, пропускаем...")
        continue

    # Загружаем нужные строки
    chunk_lines = lines[i * chunk_size: (i + 1) * chunk_size]

    start_time = time.time()  # Засекаем время

    # Сегментация
    segmented = [morpho_segment(line) for line in chunk_lines]

    # Сохраняем результат
    with open(output_path, "w", encoding="utf-8") as f_out:
        for line in segmented:
            f_out.write(line + "\n")

    # Логгируем время
    duration = time.time() - start_time
    print(f"✅ Чанк {i} обработан за {duration:.2f} сек")





✅ stop_words loaded with encoding: utf-8 | lines=189
📄 Всего чанков: 1250


⏳ Сегментирую:   0%|          | 1/1250 [00:01<28:06,  1.35s/it]

✅ Чанк 0 обработан за 1.35 сек


⏳ Сегментирую:   0%|          | 2/1250 [00:02<30:00,  1.44s/it]

✅ Чанк 1 обработан за 1.50 сек


...

✅ Чанк 1247 обработан за 1.63 сек


⏳ Сегментирую: 100%|█████████▉| 1249/1250 [41:23<00:01,  1.69s/it]

✅ Чанк 1248 обработан за 1.66 сек


⏳ Сегментирую: 100%|██████████| 1250/1250 [41:25<00:00,  1.99s/it]

✅ Чанк 1249 обработан за 1.79 сек
